In [1]:
import pandas as pd
from torch.utils.data import DataLoader,Dataset, Subset
import numpy as np
import tft_model
from data_formatters import ts_dataset  
import data_formatters.base
import expt_settings.configs
import importlib
from data_formatters import utils
import torch.optim as optim


In [2]:
pd.set_option('max_columns', 1000)

In [17]:
importlib.reload(tft_model)
importlib.reload(utils)

<module 'data_formatters.utils' from '/Users/matthewsherar/Developer/Temporal Fusion Transformer/data_formatters/utils.py'>

In [4]:
ExperimentConfig = expt_settings.configs.ExperimentConfig

config = ExperimentConfig('electricity', 'outputs')
data_formatter = config.make_data_formatter()


print("*** Training from defined parameters for {} ***".format('electricity'))
data_csv_path = 'data/hourly_electricity.csv'
print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration(
)

# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()

*** Training from defined parameters for electricity ***
Loading & splitting data...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Formatting train-valid-test splits.
Setting scalers with training data...


In [5]:
len(train.id.unique())

369

In [6]:
id_col = 'categorical_id'
time_col='hours_from_start'
input_cols =['power_usage', 'hour', 'day_of_week', 'hours_from_start', 'categorical_id']
target_col = 'power_usage'
time_steps=192
num_encoder_steps = 168
output_size = 1
max_samples = 5000
input_size = 5

elect = ts_dataset.TSDataset(id_col, time_col, input_cols,
                      target_col, time_steps, max_samples,
                     input_size, num_encoder_steps, output_size, train)

Getting valid sampling locations.
Extracting 5000 samples...


In [7]:
batch_size=64
loader = DataLoader(
            elect,
            batch_size=batch_size,
            num_workers=2,
            shuffle=True
        )

In [8]:
for batch in loader:
    break

In [21]:
static_cols = ['meter']
categorical_cols = ['hour']
real_cols = ['power_usage','hour', 'day']
config = {}
config['static_variables'] = len(static_cols)
config['time_varying_categoical_variables'] = 1
config['time_varying_real_variables_encoder'] = 4
config['time_varying_real_variables_decoder'] = 3
config['num_masked_series'] = 1
config['static_embedding_vocab_sizes'] = [369]
config['time_varying_embedding_vocab_sizes'] = [369]
config['embedding_dim'] = 16
config['lstm_hidden_dimension'] = 160
config['lstm_layers'] = 2
config['dropout'] = 0.05
config['device'] = 'cpu'
config['batch_size'] = 64
config['encode_length'] = 168
config['attn_heads'] = 4
config['num_quantiles'] = 3
config['vailid_quantiles'] = [0.1,0.5,0.9]

In [33]:
model = tft_model.TFT(config)

In [38]:
output,encoder_output, decoder_output, attn, attn_output_weights = model.forward(batch)

In [39]:
output.shape

torch.Size([64, 24, 3])

In [40]:
q_loss_func = tft_model.QuantileLoss([0.1,0.5,0.9])

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.0001)
model.train()
epochs=15
losses = []
for i in range(epochs):
    epoch_loss = [] 
    j=0
    for batch in loader:
        output, encoder_ouput, decoder_output, attn, attn_weights = model(batch)
        loss= q_loss_func(output[:,:,:].view(-1,3), batch['outputs'][:,:,0].flatten().float())
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        j+=1
        if j>10:
            break
    losses.append(np.mean(epoch_loss))
    print(np.mean(epoch_loss))
    

0.9307915351607583
